In [2]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/'My Drive'/'All_Repos'/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import glob
%matplotlib inline

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1761280/45929032 bytes (3.8%)5791744/45929032 bytes (12.6%)9814016/45929032 bytes (21.4%)13811712/45929032 bytes (30.1%)17743872/45929032 bytes (38.6%)21667840/45929032 bytes (47.2%)25616384/45929032 bytes (55.8%)29630464/45929032 bytes (64.5%)32890880/45929032 bytes (71.6%)36855808/45929032 bytes (80.2%)40697856/45929032 bytes (88.6%)44556288/45929032 bytes (97.0%)

In [8]:


images = glob.glob('camera_cal/calibration*.jpg')
objpoints = []
imgpoints = []
  
objp = np.zeros((6*9,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
for image in images:
  img = cv2.imread(image)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
  ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
  if ret == True:
    imgpoints.append(corners)
    objpoints.append(objp)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

class Line:
  
  def __init__(self):
    
    pass
  
    
def undistort_perspective_transform(img,mtx, dist,vertices,offset):

    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    img_size = (img.shape[1], img.shape[0])
    print(img_size)
    leftupperpoint  = [568,470]
    rightupperpoint = [717,470]
    leftlowerpoint  = [260,680]
    rightlowerpoint = [1100,680]
    
    src = np.float32([leftupperpoint,leftlowerpoint,
                     rightupperpoint,rightlowerpoint])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result 
        # again, not exact, but close enough for our purposes
    dst = np.float32([[200,0], [200,680], [1100,0], [1100,680]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(undist, M, img_size)
    #plt.imshow(warped)
    #plt.show()
    return warped
    
  
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    #gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=sobel_kernel)
        scaled = np.abs(255*sobelx/np.max(sobelx)).astype('uint8')
    elif orient == 'y':
        sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=sobel_kernel)
        scaled = np.abs(255*sobely/np.max(sobely)).astype('uint8')
    grad_binary = np.zeros_like(scaled)
    grad_binary[(scaled > thresh[0]) & (scaled <= thresh[1])] = 1
    
    return grad_binary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255),dir_thresh=(0.7,1.3)):
    # Calculate gradient magnitude
    # Apply threshold
    #gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    orient = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[((gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1]))] = 1

    return binary_output
  
def dir_threshold(image, sobel_kernel=3, dir_thresh=(0.66, 1.7)):
    # Calculate gradient direction
    # Apply threshold
    img = np.copy(image)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0,ksize=sobel_kernel) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    sobely = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobely = np.absolute(sobely)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    orient = np.arctan2(abs_sobely,abs_sobelx)
      
    # Threshold orient 
    
    orntbinary = np.zeros_like(scaled_sobel)
    orntbinary[(orient > dir_thresh[0]) & (orient < dir_thresh[1])] = 1
    
    return orient 
    
def pipeline(img, sobel_kernel=9,s_thresh=(90, 255), sx_thresh=(0, 255),sy_thresh=(20, 100),dir_thresh=(0,np.pi/2)):
    img = np.copy(img)
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    
    thresh = (100, 255)
    binary = np.zeros_like(R)
    binary[(R > thresh[0]) & (R <= thresh[1])] = 1
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold x gradient
    
    sxbinary = abs_sobel_thresh(s_binary, orient='x', sobel_kernel=sobel_kernel, thresh=sx_thresh)
    
    
    # Threshold orient 
    
    orntbinary = dir_threshold(img, sobel_kernel=sobel_kernel, dir_thresh=dir_thresh)
    
    
    
    
    
    combined = np.zeros_like(s_binary)
    combined[((binary == 1) & (sxbinary == 1)) | ((s_binary == 1) & (orntbinary == 1))] = 1
    #combined[((sxbinary == 1)) | ((s_binary == 1) & (orntbinary == 1))] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(combined), combined, combined)) * 255
    #color_binary = np.dstack(( orntbinary, sxbinary, s_binary)) * 255
    return color_binary
  


def roi(img,vertices):
  
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img,img, mask)
    return masked_image

def find_lane_lines(image):
  
  #grad_binary_x = abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(50, 255))
  #plt.imshow(grad_binary_x,cmap='gray')
  #plt.show()
  #mag_threshold = mag_thresh(image, sobel_kernel=9, mag_thresh=(30, 100),dir_thresh=(0.7,1.3))
  #print(image.shape)
  
  offset = 450
  vertices = np.array([[[200,image.shape[0]-50],[568,470],[717,470],[1100,image.shape[0]-50]]],dtype='int32')
  result = roi(image,vertices)
  result = undistort_perspective_transform(result,mtx, dist,vertices,offset)
  #result = pipeline(result)
  
  
  return result
  
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
  #plt.imshow(image)
  #plt.show()
  
  result = find_lane_lines(image)
  
  return result

  
def main():
  white_output = 'withPerspective.mp4'
  clip = VideoFileClip('/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Advanced-Lane-Lines/project_video.mp4')
  edge_clip = clip.fl_image(process_image)
  #[process_image(frame) for frame in clip.iter_frames()]
  %time edge_clip.write_videofile(white_output, audio=False)
    
  
    
    #result = find_lane_pipeline(image)
    #return result

if __name__ == '__main__':
  main()

(1280, 720)
[MoviePy] >>>> Building video withPerspective.mp4
[MoviePy] Writing video withPerspective.mp4



  0%|          | 2/1261 [00:00<01:05, 19.29it/s]

(1280, 720)
(1280, 720)
(1280, 720)
(1280, 720)
(1280, 720)


  1%|          | 8/1261 [00:00<01:01, 20.35it/s]


(1280, 720)
(1280, 720)
(1280, 720)
(1280, 720)


...


(1280, 720)



100%|█████████▉| 1257/1261 [02:26<00:00,  9.19it/s]

(1280, 720)
(1280, 720)
(1280, 720)



100%|█████████▉| 1260/1261 [02:27<00:00,  8.57it/s]

(1280, 720)
(1280, 720)
[MoviePy] Done.
[MoviePy] >>>> Video ready: withPerspective.mp4 

CPU times: user 1min 14s, sys: 11.1 s, total: 1min 25s
Wall time: 2min 29s
